# Execution device interoperability

This notebook demonstrates the `CPU/GPU interoperability feature`.

In [1]:
import pickle
import cuml
from cuml.common.device_selection import DeviceType, using_device_type
from cuml.neighbors import NearestNeighbors
from cuml.linear_model import LinearRegression
from cuml.datasets import make_regression, make_blobs
from cuml.model_selection import train_test_split

X_blobs, y_blobs = make_blobs(n_samples=2000, n_features=20)
X_train_blobs, X_test_blobs, y_train_blobs, y_test_blobs = train_test_split(X_blobs, y_blobs, test_size=0.2, shuffle=True)

X_reg, y_reg = make_regression(n_samples=2000, n_features=20)
X_train_reg, X_test_reg, y_train_reg, y_tes_reg = train_test_split(X_reg, y_reg, test_size=0.2, shuffle=True)

Don't have a GPU at disposal at the moment? You can work on prototyping and run estimators in CPU-mode.

In [2]:
nn = NearestNeighbors()
with using_device_type('cpu'):
    nn.fit(X_train_blobs)
    nearest_neighbors = nn.kneighbors(X_test_blobs)

Willing to deploy a ML service on a CPU-only server? Do the compute intensive training on GPU and run the inferences on a cheaper CPU server.

First, train the estimator on device and save it to disk.

In [3]:
lin_reg = LinearRegression()
with using_device_type('gpu'):
    lin_reg.fit(X_train_reg, y_train_reg)

pickle.dump(lin_reg, open("lin_reg.pkl", "wb"))
del lin_reg

Then, on the server, recover the estimator and run the inference on host.

In [4]:
recovered_lin_reg = pickle.load(open("lin_reg.pkl", "rb"))
with using_device_type('cpu'):
    predictions = recovered_lin_reg.predict(X_test_reg)

The GPU/device is the default execution platform :

In [5]:
initial_device_type = cuml.global_settings.device_type
print('initial_device_type:', initial_device_type)

initial_device_type: DeviceType.device


Estimators trainings and inferences inside a `using_device_type` context will be executed according to the execution platform selected :

In [6]:
for param in ['cpu', 'host', 'gpu', 'device']:
    with using_device_type(param):
        print('using_device_type({}):'.format(param), cuml.global_settings.device_type)

using_device_type(cpu): DeviceType.host
using_device_type(host): DeviceType.host
using_device_type(gpu): DeviceType.device
using_device_type(device): DeviceType.device


The execution platform can as well be set at the global level from the `cuml.global_settings.device_type` value.

In [7]:
cuml.global_settings.device_type = DeviceType.host